# Backtesting ML Classification-Based

## Load the model

In [47]:
import pickle

In [50]:
with open(???, 'rb') as f:
    model_dt = pickle.load(f)

## Load the data

In [1]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Simulated.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,daily_diff,daily_diff_direction,buy,buy_sell
Date,,,,,,,,,
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.882095,UP,1,1
2016-12-12,56.803028,57.244073,56.711145,57.124622,20198100,0.183765,UP,1,0
...,...,...,...,...,...,...,...,...,...
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,4.649994,UP,1,0
2023-03-16,265.209991,276.559998,263.279999,276.200012,54768800,10.760010,UP,1,0


## Backtesting.py Library

### Create your Strategy Class

In [ ]:
from backtesting import ???

In [ ]:
Backtest(???)

In [40]:
class ???(Strategy):
    def init(self):
        self.model = ???

    def next(self):
        explanatory_today = ???
        forecast_tomorrow = ???

### Define initial conditions

### Run backtesting

### Interpret backtesting results

In [ ]:
results.to_frame(name='Values').loc[:'Return [%]']

### Visualize the strategy performance